# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [ ]:
# all import statements needed for the project, for example:

from math import radians, cos, sin, asin, sqrt

import bs4
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import requests
import sqlalchemy as db
import pyarrow.parquet as pq
import re
import numpy as np
import sys
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

In [ ]:
# any general notebook setup, like log formatting
!pip install pyarrow
!pip install geopandas

In [ ]:
#urls
TAXI_URL = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

#local csvs
UBER_DATA = "uber_rides_sample.csv"
WEATHER_CSV = ["2015_weather.csv","2014_weather.csv","2013_weather.csv","2012_weather.csv","2011_weather.csv","2010_weather.csv","2009_weather.csv"]

#coords
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library.
* [ ] Taxi data:
    * [ ] Use the `re` module, and the packages `requests`, BeautifulSoup (`bs4`), and (optionally) `pandas` to programmatically download the required CSV files & load into memory.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* [ ] Weather & Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
* [ ] All data:
    * [ ] Load the data using `pandas`
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * (Taxi & Uber data) Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] (Taxi data) Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
* [ ] Weather data:
    * [ ] Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
    * [ ] You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 

### Calculating distance
_**TODO:** to find the trip distance by transfering coordinates (degrees) into the distance in kilometers._

_step1: get the latitude and longitude from the coordinates and transfer from degrees to radians._

_step2: get the distance of two latitudes and longitudes from step1._

_step3: import the results from step2 into the formal equation of calculating distance._

_step4: import a new column "distance" into the dataframe with the result of step3._

_NOTE: suppose the radius of earth in kilometers is 6378._

In [ ]:
def calculate_distance(from_coord, to_coord):
    
    #step1:get the latitude and longitude from the coordinates and transfer from degrees to radians.
    pick_lat = from_coord["pickup_latitude"].map(radians)
    pick_long = from_coord["pickup_longitude"].map(radians)
    drop_lat = to_coord["dropoff_latitude"].map(radians)
    drop_long = to_coord["dropoff_longitude"].map(radians)
    
    #step2: get the distance of two latitudes and longitudes from step1.
    distance_long = pick_long - drop_long
    distance_lat = pick_lat - drop_lat
    
    #step3: import the results from step2 into the formal equation of calculating distance.
    x = ((distance_lat / 2).map(sin))**2 + (pick_lat).map(cos) * (drop_lat).map(cos) * ((distance_long / 2).map(sin))**2
    c = 2 * (x.map(sqrt)).map(asin)
    r = 6378 # Radius of earth in kilometers
    
    return c*r


In [ ]:
def add_distance_column(dataframe):
    #step4: import a new column "distance" into the dataframe with the result of step3.
    from_coord = dataframe[["pickup_latitude","pickup_longitude"]]
    to_coord = dataframe[["dropoff_latitude","dropoff_longitude"]]
    
    dataframe["distance"] = calculate_distance(from_coord, to_coord)
    dataframe["distance"] = dataframe["distance"].astype("float32")
    
    return dataframe

### Processing Taxi Data

_**TODO:** Downloand, Clean, and Sample Yellow Taxi Trip Records._

_**step1:** get source data from the website: 78 "Yellow Taxi Trip Records" parquet files (from 2009.01 to 2015.06)._

_**step2:** upzip taxi_zones file and remove the duplicated location IDs._

_**step3:** Clean data: normalize column names, remove unneccessary columns and data points by requirements, standardize the data type._
            
_**step4:** Sample data: according to the 200,000 samples in uber data, each month we select 2564 samples from the taxi data._

_**step5:** Create a gigantic dataframe combining cleaned data in every month._

In [ ]:
def find_taxi_parquet_links():
    response = requests.get(TAXI_URL)
    html = response.content
    soup = bs4.BeautifulSoup(html, "html.parser")
    yellow_a_tags = soup.find_all("a", attrs={"title": "Yellow Taxi Trip Records"})
    result = [a["href"] for a in yellow_a_tags]
    pattern = re.compile(
    r"yellow_tripdata_20((15-0([1-6]))|(09-(\d{2}))|(1[0-4]-(\d{2}))).parquet"
    )
    links = []
    for link in result:
        match = pattern.search(link)
        if match:
            links.append(match.string) 
    return links

In [ ]:
def get_and_clean_month_taxi_data(url):

    df = pd.read_parquet(url)

    taxi_zones = gpd.read_file(filename='taxi_zones.zip', engine='fiona')
    taxi_zones = taxi_zones.to_crs(4326)
    taxi_zones['longitude'] = taxi_zones.centroid.x
    taxi_zones['latitude'] = taxi_zones.centroid.y

    if 'PULocationID' in df.columns:
        df['pickup_latitude'] = df['PULocationID'].map(taxi_zones['latitude'])
        df['pickup_longitude'] = df['PULocationID'].map(taxi_zones['longitude'])
    if 'DOLocationID' in df.columns:
        df['dropoff_latitude'] = df['DOLocationID'].map(taxi_zones['latitude'])
        df['dropoff_longitude'] = df['DOLocationID'].map(taxi_zones['longitude'])
        
    # Set all column names to lower case    
    df = df.rename(columns=str.lower)
    
    # Normalize column names
    df.rename(columns={'tpep_pickup_datetime':'pickup_datetime','trip_pickup_datetime':'pickup_datetime',
                       'tpep_dropoff_datetime':'dropoff_datetime','trip_dropoff_datetime':'dropoff_datetime',
                       'start_lon':'pickup_longitude', 'start_lat':'pickup_latitude', 
                       'end_lon':'dropoff_longitude', 'end_lat':'dropoff_latitude','tip_amt':'tip_amount'}, inplace=True)
    
    # Remove invalid data points
    df.dropna(subset=['pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude','tip_amount'],inplace=True)
    df = df[df['passenger_count'] >= 1]
    df = df[(df['pickup_latitude'] != df['dropoff_latitude']) | (df['pickup_longitude'] != df['dropoff_longitude'])]
    
    # Remove unnecessary columns
    df = df[['pickup_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','tip_amount']]

    # Remove trips that start and/or end outside the designated coordinate box (NEW_YORK_BOX_COORDS)
    df = df[(df['pickup_latitude'] >= NEW_YORK_BOX_COORDS[0][0]) & (df['pickup_latitude'] <= NEW_YORK_BOX_COORDS[1][0])
            & (df['pickup_longitude'] >= NEW_YORK_BOX_COORDS[0][1]) & (df['pickup_longitude'] <= NEW_YORK_BOX_COORDS[1][1])]
    df = df[(df['dropoff_latitude'] >= NEW_YORK_BOX_COORDS[0][0]) & (df['dropoff_latitude'] <= NEW_YORK_BOX_COORDS[1][0])
            & (df['dropoff_longitude'] >= NEW_YORK_BOX_COORDS[0][1]) & (df['dropoff_longitude'] <= NEW_YORK_BOX_COORDS[1][1])]


    # Sample
    df = df.sample(n=2564)
    
    # Use appropriate type for each column
    df = df.reset_index(drop=True)
    df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
    df.iloc[:,1:7] = df.iloc[:,1:7].astype("float32")
    return df

In [ ]:
def get_and_clean_taxi_data():
    all_taxi_dataframes = []
    
    all_parquet_urls = find_taxi_parquet_links()
    for parquet_url in tqdm(all_parquet_urls):
        dataframe = get_and_clean_month_taxi_data(parquet_url)
        add_distance_column(dataframe)
        
        all_taxi_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.concat(all_taxi_dataframes)
        
    return taxi_data

### Processing Uber Data

_**TODO:** Get cleaned Uber Rides Sample records._

_**step1:** Remove unneccessary columns and data points by requirements, standardize the data type._
            
_**step2:** Create a dataframe with cleaned datapoints._

In [ ]:
def load_and_clean_uber_data(csv_file):
    df = pd.read_csv(csv_file)
    
    # Remove unnecessary columns
    df = df[['pickup_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude']]
    
    # Remove trips that start and/or end outside the designated coordinate box (NEW_YORK_BOX_COORDS)
    df = df[(df['pickup_latitude'] >= NEW_YORK_BOX_COORDS[0][0]) & (df['pickup_latitude'] <= NEW_YORK_BOX_COORDS[1][0])
            & (df['pickup_longitude'] >= NEW_YORK_BOX_COORDS[0][1]) & (df['pickup_longitude'] <= NEW_YORK_BOX_COORDS[1][1])]
    
    # Use appropriate type for each column
    df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
    df.iloc[:,1:5] = df.iloc[:,1:5].astype("float32")
    return df

In [ ]:
def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_DATA)
    add_distance_column(uber_dataframe)
    return uber_dataframe

### Processing Weather Data

_**TODO:** Get claned Weather data records by creating two dataFrames: one for required hourly data, and one for the required daily data._

_**step1:** Normalize the columns, remove unneccessary columns and data points by requirements, standardize the data type, and fillout the required blanks._
            
_**step2:** Create two required dataframes with cleaned datapoints._

In [ ]:
def clean_month_weather_data_hourly(csv_file):
    df = pd.read_csv(csv_file)
    
    #find the required columns
    df = df[["DATE","HourlyPrecipitation", "HourlyWindSpeed"]]
    
    #normalize the 'Precipitation' columns into numeric form (dtype: float32) and fillout the blank with value 0
    df["HourlyPrecipitation"].fillna(value = 0, inplace = True)
    df[df["HourlyPrecipitation"]=="T"] = float(0.00001)
    df[df["HourlyPrecipitation"]=="M"] = np.nan
    df['HourlyPrecipitation'] = pd.to_numeric(df['HourlyPrecipitation'], errors = "coerce")
    
    #fill out the missing blank for 'WindSpeed' with 0
    df["HourlyWindSpeed"].fillna(value = 0, inplace = True)
    
    #normalize the 'DATE' columns into datetime form
    df["DATE"] = pd.to_datetime(df["DATE"])
    df = df.astype({"HourlyWindSpeed":"int8","HourlyPrecipitation":"float32"})
    
    return df

In [ ]:
def clean_month_weather_data_daily(csv_file):
    df = clean_month_weather_data_hourly(csv_file)
    df['DATE'] = pd.to_datetime(df['DATE'], errors='coerce').dt.date
    df_windspeed = df.groupby('DATE')['HourlyWindSpeed'].mean()
    df_precipitation = df.groupby('DATE')['HourlyPrecipitation'].sum()
    
    df = pd.merge(df_windspeed,df_precipitation,on='DATE')
    df = df.rename(columns={'HourlyWindSpeed': 'DailyWindSpeed', 'HourlyPrecipitation': 'DailyPrecipitation'})
    df = df.astype({"DailyWindSpeed":"int8","DailyPrecipitation":"float32"})
    return df

In [ ]:
def load_and_clean_weather_data():
    hourly_dataframes = []
    daily_dataframes = []
    
    # add some way to find all weather CSV files
    # or just add the name/paths manually
    weather_csv_files = ["2015_weather.csv","2014_weather.csv","2013_weather.csv","2012_weather.csv","2011_weather.csv","2010_weather.csv","2009_weather.csv"]
    
    for csv_file in tqdm(weather_csv_files):
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

### Process All Data

_This is where you can actually execute all the required functions._

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
taxi_data = get_and_clean_taxi_data()

In [ ]:
taxi_data.sort_values('distance')

In [ ]:
uber_data = get_uber_data()

In [ ]:
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

## Part 2: Storing Cleaned Data

_Write some prose that tells the reader what you're about to do here._

In [ ]:
engine = db.create_engine(DATABASE_URL)

In [ ]:
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather
(
    id INTEGER PRIMARY KEY,
    DATE DATE,
    HourlyWindSpeed INTETER,
    HourlyPrecipitation FLOAT
)
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather
(
    id INTEGER PRIMARY KEY,
    DATE DATE,
    DailyWindSpeed INTETER,
    DailyPrecipitation FLOAT
)
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi_trips
(
    id INTEGER PRIMARY KEY,
    pickup_datetime DATE,
    PULocationID INTETER
    DOLocationID INTETER
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT,
    distance FLOAT
)
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips
(
    id INTEGER PRIMARY KEY,
    pickup_datetime DATE,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT,
    distance FLOAT
)
"""

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [ ]:
# create the tables with the schema files
with engine.connect() as connection:
    pass

### Add Data to Database

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
def write_dataframes_to_table(table_to_df_dict):
    for table, df in table_to_df_dict.items():
        df.to_sql(table, con=engine, if_exists="append", index=False)

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_weather_data,
    "daily_weather": daily_weather_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

In [ ]:
def write_query_to_file(query, outfile):
    with open(QUERY_DIRECTORY + "/" + outfile, "w") as f:
        f.write(query)

### Query #1

_**TODO:** Find the most popular hour of the day to take a yellow taxi from 01-2009 to 06-2015._

In [ ]:
QUERY_1 = """
SELECT strftime("%H", pickup_datetime) AS hours,
COUNT(*) AS hour_frequency
FROM taxi_trips
GROUP BY hours
ORDER BY hour_frequency desc
"""

In [ ]:
engine.execute(QUERY_1).fetchall()

In [ ]:
write_query_to_file(QUERY_1, "popular_hours_of_taxi.sql")

### Query #2

_**TODO:** Find the most popular day of the week to take an uber from 01-2009 to 06-2015._

In [ ]:
QUERY_2 = """
SELECT strftime("%w", pickup_datetime) AS days,
COUNT(*) AS day_frequency
FROM uber_trips
GROUP BY days
ORDER BY day_frequency DESC;

"""

In [ ]:
engine.execute(QUERY_2).fetchall()

In [ ]:
write_query_to_file(QUERY_2, "popular_days_of_uber.sql")

### Query #3

_**TODO:** Find the 95% percentile of distance traveled for all hired trips during July 2013._

In [ ]:
QUERY_3 = """
WITH hired_trips AS
(
    SELECT pickup_datetime, distance FROM taxi_trips
    WHERE pickup_datetime BETWEEN "2013-07-01 00:00:00" AND "2013-07-31 23:59:59"
    UNION
    SELECT pickup_datetime, distance FROM uber_trips
    WHERE pickup_datetime BETWEEN "2013-07-01 00:00:00" AND "2013-07-31 23:59:59"
)

SELECT distance
FROM hired_trips
ORDER BY distance desc

LIMIT 1
OFFSET (SELECT COUNT(*) FROM hired_trips) * 5/100 

"""

In [ ]:
engine.execute(QUERY_3).fetchall()

In [ ]:
write_query_to_file(QUERY_3, "95_percentile_of_hired_trips.sql")

### Query #4

_**TODO 1:** Find the top 10 days with the highest number of hired rides for 2009._

_**TODO 2:** Find the average distance for each day._

In [ ]:
QUERY_4 = """
WITH hired_trips AS
(
    SELECT pickup_datetime, distance FROM taxi_trips
    WHERE pickup_datetime BETWEEN "2009-01-01 00:00:00" AND "2009-12-31 23:59:59"
    UNION
    SELECT pickup_datetime, distance FROM uber_trips
    WHERE pickup_datetime BETWEEN "2009-01-01 00:00:00" AND "2009-12-31 23:59:59"
)

SELECT DATE(pickup_datetime) AS dates,
AVG(distance) AS average_distance,
COUNT(*) AS number_of_hired_rides

FROM hired_trips

GROUP BY dates
ORDER BY number_of_hired_rides desc

LIMIT 10


"""

In [ ]:
engine.execute(QUERY_4).fetchall()

In [ ]:
write_query_to_file(QUERY_4, "top_ten_days_of_hired_trips.sql")

### Query #5

_**TODO 1:** Find the windiest 10 days in 2014._

_**TODO 2:** Find the number of hired trips on those windiest 10 days._

In [ ]:
QUERY_5 = """
WITH hired_trips AS
(
    SELECT DATE(pickup_datetime) AS dates 
    COUNT(*) AS number_of_hired_trips
    FROM taxi_trips
    WHERE pickup_datetime BETWEEN "2014-01-01 00:00:00" AND "2014-12-31 23:59:59"
    GROUP BY dates
    
    UNION
    
    SELECT DATE(pickup_datetime) AS dates 
    COUNT(*) AS number_of_hired_trips
    FROM uber_trips
    WHERE pickup_datetime BETWEEN "2014-01-01 00:00:00" AND "2014-12-31 23:59:59"
    GROUP BY dates
)

HAVING dates IN(
SELECT dates(DATE)
FROM daily_weather
WHERE DATE BETWEEN "2014-01-01 00:00:00" AND "2014-12-31 23:59:59"
ORDER BY DailyWindSpeed desc
LIMIT 10)

"""

In [ ]:
engine.execute(QUERY_5).fetchall()

In [ ]:
write_query_to_file(QUERY_5, "10_windiest_days.sql")

### Query #6

_**TODO 1:** Find the number of trips taken each hour from Oct 29, 2012 to Oct 30, 2012, and the week leading up to it,._

_**TODO 2:** Find the amount of precipitation and the sustained wind speed for each hour from above days._

In [ ]:
QUERY_6 = """


"""

In [ ]:
engine.execute(QUERY_5).fetchall()

In [ ]:
write_query_to_file(QUERY_6, "trips_precipitation_windspeed.sql")

## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization 1

_**TODO:** Create an appropriate visualization for the first query/question in part 3._

In [ ]:
def popular_hours(dataframe):
    plot_title = "Total Number of Taxi Trips(Hours)"
    x_label = "Hours"
    y_label = "Total Number of Taxi Trips"
    dataframe.plot(kind="bar",x="hours", y="hour_frequency", title=plot_title, xlabel=x_label, ylabel=y_label)


In [ ]:
def get_data_for_visual_1():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    dataframe = pd.read_sql_query(QUERY_1, engine)
    df = dataframe.sort_values(by = "hours", ascending = True)
    return df

In [ ]:
some_dataframe = get_data_for_visual_1()
popular_hours(some_dataframe)

### Visualization 2

_**TODO 1:** Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month)._

_**TODO 2:** Include the 90% confidence interval around the mean in the visualization._


In [ ]:
def distance_per_month(dataframe):
    plot_title = "Average Distance Travelled(Month)"
    x_label = "Month"
    y_label = "Average Distance"
    dataframe.plot(kind="bar",x="month", y="average_distance", title=plot_title, xlabel=x_label, ylabel=y_label)

In [ ]:
def get_data_for_visual_2():
    # Query SQL database for the data needed.
    QUERY_avg = """
    WITH hired_trips AS
    (
    SELECT pickup_datetime, distance FROM taxi_trips
    UNION
    SELECT pickup_datetime, distance FROM uber_trips
    )
    
    SELECT strftime("%m", pickup_datetime) AS month,
    AVG(distance_by_month) AS average_distance
    
    FROM (
    SELECT strftime("%Y-%m",pickup_datetime) AS Yr_Month,
    SUM(distance) AS distance_by_month
    FROM hired_trips
    GROUP BY Yr_Month)
    
    GROUP BY Month
    
    """
    dataframe = pd.read_sql_query(QUERY_avg, engine)
    return dataframe

In [ ]:
some_dataframe = get_data_for_visual_2()
distance_per_month(some_dataframe)